In [3]:
import os
import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt
from PIL import Image
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image
import tensorflow as tf

ModuleNotFoundError: No module named 'cv2'

In [10]:
# Load MobileNetV2 model for skin detection
skin_model = MobileNetV2(weights='imagenet')

In [11]:
def is_skin_image(img_path):
    """Checks if the given image contains skin before classifying cancer."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    preds = skin_model.predict(img_array)
    decoded_preds = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=3)[0]
    
    skin_related_classes = {'skin', 'epidermis', 'dermatology', 'human'}
    for _, label, _ in decoded_preds:
        if label.lower() in skin_related_classes:
            return True
    return False

In [ ]:
def display_image(image_path, title="Input Image"):
    """Displays the given image."""
    img = Image.open(image_path)
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()

In [12]:
def read_image(image_path, target_size=(100, 100)):
    """Reads an image, converts to grayscale, and resizes it."""
    img = Image.open(image_path).convert("L")  
    img = img.resize(target_size)  
    return np.array(img, dtype=np.uint8)

In [13]:
# ----------------- Step 4: Feature Extraction -----------------
def extract_contourlet_features(image):
    """Extracts contourlet (wavelet-based) features."""
    if len(image.shape) != 2:  
        raise ValueError(f"Invalid image shape: {image.shape}. Expected 2D grayscale.")
    
    coeffs = pywt.wavedec2(image, 'db1', level=3)  
    features = []
    for coeff in coeffs:
        if isinstance(coeff, tuple):
            for subband in coeff:
                features.append(np.mean(subband))
                features.append(np.var(subband))
        else:
            features.append(np.mean(coeff))
            features.append(np.var(coeff))
    return features

In [14]:
def extract_lbp_features(image, num_points=24, radius=3):
    """Extracts LBP (Local Binary Pattern) features."""
    lbp = local_binary_pattern(image, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    return hist.astype("float") / hist.sum()

In [8]:
def extract_features(image):
    """Combines contourlet and LBP features."""
    contourlet_features = extract_contourlet_features(image)
    lbp_features = extract_lbp_features(image)
    return np.concatenate((contourlet_features, lbp_features))

In [15]:
# ----------------- Step 5: Load Dataset -----------------
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = read_image(img_path)
            images.append(extract_features(img))
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    return np.array(images, dtype="float")

In [16]:
# Define dataset paths
folder_benign_train = r'C:\Users\HP\Desktop\Skin cancer detection_mini project\data\train\benign'
folder_malignant_train = r'C:\Users\HP\Desktop\Skin cancer detection_mini project\data\train\malignant'
folder_benign_test = r'C:\Users\HP\Desktop\Skin cancer detection_mini project\data\test\benign'
folder_malignant_test = r'C:\Users\HP\Desktop\Skin cancer detection_mini project\data\test\malignant'


In [17]:
# Load dataset
X_benign = load_images_from_folder(folder_benign_train)
X_malignant = load_images_from_folder(folder_malignant_train)
X_benign_test = load_images_from_folder(folder_benign_test)
X_malignant_test = load_images_from_folder(folder_malignant_test)

In [18]:
# Create labels (0 = Benign, 1 = Malignant)
y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])
y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])

In [19]:
# Merge Data
X_train = np.concatenate((X_benign, X_malignant), axis=0)
y_train = np.concatenate((y_benign, y_malignant), axis=0)
X_test = np.concatenate((X_benign_test, X_malignant_test), axis=0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis=0)

In [20]:
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", svm_acc)

SVM Accuracy: 0.7818181818181819


In [21]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", rf_acc)

Random Forest Accuracy: 0.8227272727272728


In [22]:
# Choose Best Model
best_model = svm_model if svm_acc > rf_acc else rf_model
print("Best Model Selected:", "SVM" if best_model == svm_model else "Random Forest")

Best Model Selected: Random Forest


In [ ]:
def predict_skin_cancer(image_path, healthy_threshold=0.55):
    """
    Predicts skin cancer type (Benign/Malignant/No Cancer) for a given image.
    If the confidence score for both benign and malignant is low, classify as "No Cancer".
    """
    display_image(image_path, "Input Image")

    if not is_skin_image(image_path):
        print("This is not a skin image. Please provide a valid skin image.")
        return "Not a skin image"
    
    img = read_image(image_path)  
    features = extract_features(img)  
    features = np.array(features).reshape(1, -1)

    pred_prob = best_model.predict_proba(features)[0]
    pred_label = best_model.predict(features)[0]

    benign_prob, malignant_prob = pred_prob[0], pred_prob[1]
    
    print(f"Prediction Confidence: Benign: {benign_prob:.2f}, Malignant: {malignant_prob:.2f}")

    if benign_prob < healthy_threshold and malignant_prob < healthy_threshold:
        return "No Cancer (Healthy Skin)"

    return "Benign" if pred_label == 0 else "Malignant"

In [ ]:
# ----------------- Step 8: Test the Model -----------------
sample_image = r'C:\Users\HP\Desktop\Skin cancer detection_mini project\data\image1.jpg'
print("Prediction:", predict_skin_cancer(sample_image))